# ConversationTokenBufferMemory

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/05-Memory/03-ConversationTokenBufferMemory.ipynb)[![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/05-Memory/03-ConversationTokenBufferMemory.ipynb)

## 概覽

```ConversationTokenBufferMemory``` 在緩衝記憶體中存儲最近的對話歷史，並根據**token長度**而非對話次數來決定何時清除對話內容。

主要參數：
- ```max_token_limit```：設定存儲對話內容的最大令牌長度
- ```return_messages```：為 True 時，以聊天格式返回訊息。為 False 時，返回字串
- ```human_prefix```：在人類訊息前添加的前綴（預設："Human"）
- ```ai_prefix```：在 AI 訊息前添加的前綴（預設："AI"）

## 目錄

- [概覽](#概覽)
- [環境設定](#環境設定)
- [將最大令牌長度限制為 50](#將最大令牌長度限制為-50)
- [將最大令牌長度設定為 150](#將最大令牌長度設定為-150)

## 參考資料

- [LangChain Python API Reference > langchain: 0.3.13 > memory > ConversationTokenBufferMemory](https://python.langchain.com/api_reference/langchain/memory/langchain.memory.token_buffer.ConversationTokenBufferMemory.html)

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- ```langchain-opentutorial``` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain-anthropic",
        "langchain_community",
        "langchain_text_splitters",
        "langchain_openai",
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "ConversationTokenBufferMemory",
    }
)

Environment variables have been set successfully.


You can alternatively set ```OPENAI_API_KEY``` in ```.env``` file and load it.

[Note] This is not necessary if you've already set ```OPENAI_API_KEY``` in previous steps.

In [4]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

## 將最大令牌長度限制為 50

本節演示如何將對話記憶體限制為 50 個令牌

**解釋：**

這個部分主要是要展示當我們設定 `max_token_limit=50` 時，`ConversationTokenBufferMemory` 會如何運作。當對話歷史累積的令牌數量超過 50 個時，系統會自動刪除最舊的對話內容，只保留最新的對話，確保總令牌數不超過設定的限制。這種機制有助於：

1. **控制記憶體使用量**：避免對話歷史無限增長
2. **符合模型限制**：確保不超過語言模型的上下文窗口
3. **維持對話連貫性**：保留最相關的近期對話內容
4. **節省計算成本**：減少需要處理的令牌數量

這種基於令牌長度的管理方式比單純計算對話輪次更精確，因為不同的訊息可能包含不同數量的令牌。

In [5]:
from langchain.memory import ConversationTokenBufferMemory
from langchain_openai import ChatOpenAI


# Create LLM model
llm = ChatOpenAI(model_name="gpt-4o-mini")

# Configure memory
memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=50,
    return_messages=True,  # Limit maximum token length to 50
)

/var/folders/1h/lrydr5_50zx8thkmv_czrbzc0000gn/T/ipykernel_59195/361976439.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationTokenBufferMemory(


In [6]:
# Add arbitrary conversations
memory.save_context(
    inputs={
        "human": "Hello, I recently purchased a machine tool from your company. Could you tell me how to install it?"
    },
    outputs={
        "ai": "Hello! Thank you for your purchase. Could you please tell me the machine model number?"
    },
)
memory.save_context(
    inputs={"human": "Yes, the model number is XG-200."},
    outputs={
        "ai": "Thank you. I'll help you with the installation guide for the XG-200 model. First, please check the power supply status at the installation site. The machine requires 220V power."
    },
)
memory.save_context(
    inputs={"human": "I've checked the power. What's the next step?"},
    outputs={
        "ai": "Good. Next, please place the machine on a flat and stable surface. Then, proceed with cable connections according to the provided user manual."
    },
)
memory.save_context(
    inputs={"human": "How do I make the connections?"},
    outputs={
        "ai": "Please refer to page 5 of the manual. There are detailed instructions for cable connections. If you have any difficulties with this process, I'll be happy to help further."
    },
)
memory.save_context(
    inputs={"human": "What should I do after the installation is complete?"},
    outputs={
        "ai": "Once the installation is complete, please turn on the power and perform the initial operation test. The test procedure is explained on page 10 of the manual. If there are any issues with the machine or if you need additional support, please don't hesitate to contact us."
    },
)
memory.save_context(
    inputs={"human": "Thank you, this has been very helpful!"},
    outputs={
        "ai": "We're always ready to help. If you have any additional questions or need support, please feel free to ask. Have a great day!"
    },
)

In [7]:
# Check the conversation history
memory.load_memory_variables({})["history"]

[HumanMessage(content='Thank you, this has been very helpful!', additional_kwargs={}, response_metadata={}),
 AIMessage(content="We're always ready to help. If you have any additional questions or need support, please feel free to ask. Have a great day!", additional_kwargs={}, response_metadata={})]

## Setting Maximum Token Length to 150

Let's check how the conversation is stored when we set the maximum token length to **150**.

In [8]:
# Memory configuration
memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,  # Limit maximum token length to 150
)

In [9]:
# Add arbitrary conversations
memory.save_context(
    inputs={
        "human": "Hello, I recently purchased a machine tool from your company. Could you tell me how to install it?"
    },
    outputs={
        "ai": "Hello! Thank you for your purchase. Could you please tell me the machine model number?"
    },
)
memory.save_context(
    inputs={"human": "Yes, the model number is XG-200."},
    outputs={
        "ai": "Thank you. I'll help you with the installation guide for the XG-200 model. First, please check the power supply status at the installation site. The machine requires 220V power."
    },
)
memory.save_context(
    inputs={"human": "I've checked the power. What's the next step?"},
    outputs={
        "ai": "Good. Next, please place the machine on a flat and stable surface. Then, proceed with cable connections according to the provided user manual."
    },
)
memory.save_context(
    inputs={"human": "How do I make the connections?"},
    outputs={
        "ai": "Please refer to page 5 of the manual. There are detailed instructions for cable connections. If you have any difficulties with this process, I'll be happy to help further."
    },
)
memory.save_context(
    inputs={"human": "What should I do after the installation is complete?"},
    outputs={
        "ai": "Once the installation is complete, please turn on the power and perform the initial operation test. The test procedure is explained on page 10 of the manual. If there are any issues with the machine or if you need additional support, please don't hesitate to contact us."
    },
)
memory.save_context(
    inputs={"human": "Thank you, this has been very helpful!"},
    outputs={
        "ai": "We're always ready to help. If you have any additional questions or need support, please feel free to ask. Have a great day!"
    },
)

In [10]:
# Check the conversation history
memory.load_memory_variables({})["history"]

[HumanMessage(content='What should I do after the installation is complete?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Once the installation is complete, please turn on the power and perform the initial operation test. The test procedure is explained on page 10 of the manual. If there are any issues with the machine or if you need additional support, please don't hesitate to contact us.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thank you, this has been very helpful!', additional_kwargs={}, response_metadata={}),
 AIMessage(content="We're always ready to help. If you have any additional questions or need support, please feel free to ask. Have a great day!", additional_kwargs={}, response_metadata={})]